In [ ]:
#####Generalized Factorization Machines#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [ ]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, pattern):
    if pattern==1:
        z_id = np.array(np.argmax(np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis], axis=1), dtype="int")
        Z = np.diag(np.repeat(1, k))[z_id, ]
        return z_id, Z
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    return z_id

In [ ]:
###データの生成####
##データの設定
s = 3
k1 = 10
k2 = 10
k3 = 7
hh = 3500
item = 3000
element = 200
pt1 = np.random.poisson(50.0, hh)
pt2 = np.random.poisson(20.0, hh)
hhpt1 = np.sum(pt1)
hhpt2 = np.sum(pt2)
k_vec1 = np.repeat(1, k1)
k_vec2 = np.repeat(1, k2)
k_vec3 = np.repeat(1, k3)

In [ ]:
##IDとインデックスの設定
#IDの設定
d_id1 = np.array(np.repeat(np.arange(hh), pt1), dtype="int16")
d_id2 = np.array(np.repeat(np.arange(hh), pt2), dtype="int16")
pt_id1 = np.array(list(itertools.chain(*[np.array(range(pt1[i]), dtype="int") for i in range(hh)])), dtype="int16")
pt_id2 = np.array(list(itertools.chain(*[np.array(range(pt2[i]), dtype="int") for i in range(hh)])), dtype="int16")

#インデックスの設定
d_list1 = [i for i in range(hh)]
d_list2 = [i for i in range(hh)]
for i in range(hh):
    d_list1[i] = np.array(np.where(d_id1==i)[0], dtype="int")
    d_list2[i] = np.array(np.where(d_id2==i)[0], dtype="int")

In [ ]:
##アイテムを生成
#レコードごとの頻度を生成
gamma01 = np.random.gamma(1.5, 1/0.5, hh)[d_id1]
gamma02 = np.random.gamma(1.5, 1/0.5, hh)[d_id1]
freq1 = np.random.poisson(gamma01, hhpt1)
freq2 = np.random.poisson(gamma02, hhpt1)
freq1[freq1 < 1] = 1; freq2[freq2 < 1] = 1
max_freq1 = np.max(freq1)
max_freq2 = np.max(freq2)

#トピックのパラメータを生成
topic = 25
theta0 = np.random.dirichlet(np.repeat(0.5, topic), hh)
phi_item = np.random.dirichlet(np.repeat(0.5, item), topic)
phi_element = np.random.dirichlet(np.repeat(0.5, element), topic)
z2 = np.array(rmnom(theta0[d_id2, ], hhpt2, topic, 0), dtype="int")

#トピックとアイテムの格納用配列
z11 = np.full((hhpt1, max_freq1), topic, dtype="int16")
z12 = np.full((hhpt1, max_freq2), topic, dtype="int16")
item_id11 = np.full((hhpt1, max_freq1), item, dtype="int16")
item_id12 = np.full((hhpt1, max_freq2), item, dtype="int16")
item_id2 = np.array(np.repeat(0, hhpt2), dtype="int16")

#多項分布からアイテムを生成
for i in range(hhpt1):
    index1 = np.arange(freq1[i])
    index2 = np.arange(freq2[i])
    z11[i, index1] = np.array(rmnom(theta0[d_id1[np.repeat(i, freq1[i])], ], freq1[i], topic, 0), dtype="int16")
    z12[i, index2] = np.array(rmnom(theta0[d_id1[np.repeat(i, freq2[i])], ], freq2[i], topic, 0), dtype="int16")
    item_id11[i, index1] = np.array(rmnom(phi_item[z11[i, index1], ], freq1[i], item, 0), dtype="int16")
    item_id12[i, index2] = np.array(rmnom(phi_item[z12[i, index2], ], freq2[i], item, 0), dtype="int16")
    
for i in range(hh):
    item_id2[d_list2[i]] = np.array(rmnom(phi_item[z2[d_list2[i]], ], pt2[i], item, 0), dtype="int16")

In [ ]:
##エレメントを生成
#エレメントの格納用配列
element_id11 = np.full((hhpt1, max_freq1), item, dtype="int16")
element_id12 = np.full((hhpt1, max_freq2), item, dtype="int16")
element_id2 = np.array(np.repeat(0, hhpt2), dtype="int16")

#多項分布からエレメントを生成
for i in range(hhpt1):
    index1 = np.arange(freq1[i])
    index2 = np.arange(freq2[i])
    element_id11[i, index1] = np.array(rmnom(phi_element[z11[i, index1], ], freq1[i], element, 0), dtype="int16")
    element_id12[i, index2] = np.array(rmnom(phi_element[z12[i, index2], ], freq2[i], element, 0), dtype="int16")
    
for i in range(hh):
    element_id2[d_list2[i]] = np.array(rmnom(phi_element[z2[d_list2[i]], ], pt2[i], element, 0), dtype="int16")

In [ ]:
##レコード単位に拡張したidとインデックスを作成
#レコード単位のidを作成
N1 = np.sum(freq1)
N2 = np.sum(freq2)
record_id1 = np.repeat(np.arange(hhpt1), freq1)
record_id2 = np.repeat(np.arange(hhpt1), freq2)

#拡張したidを作成
d_long1 = np.repeat(d_id1, freq1); d_long2 = np.repeat(d_id1, freq2)
item_long1 = item_id11[item_id11 < item]; item_long2 = item_id12[item_id12 < item]
element_long1 = element_id11[element_id11 < element]; element_long2 = element_id12[element_id12 < element]

#idのフラッグを作成
Zi1 = sparse.coo_matrix((np.repeat(1, N1), (np.arange(N1), record_id1)), shape=(N1, hhpt1)).tocsr()
Zi2 = sparse.coo_matrix((np.repeat(1, N2), (np.arange(N2), record_id2)), shape=(N2, hhpt1)).tocsr()

#レコードのインデックスを作成
record_list1 = [i for i in range(hhpt1)]
record_list2 = [i for i in range(hhpt1)]
record_index1 = [i for i in range(hhpt1)]
record_index2 = [i for i in range(hhpt1)]
for i in range(hhpt1):
    record_list1[i] = np.array(np.where(record_id1==i)[0], dtype="int")
    record_list2[i] = np.array(np.where(record_id2==i)[0], dtype="int")
    record_index1[i] = np.array(np.arange(freq1[i]), dtype="int8")
    record_index2[i] = np.array(np.arange(freq2[i]), dtype="int8")

In [ ]:
##アイテムインデックスを作成
#インデックスの格納用配列
item_list11 = [i for i in range(item)]
item_list12 = [i for i in range(item)]
item_list2 = [i for i in range(item)]
item_row10 = [i for i in range(item)]
item_row11 = [i for i in range(item)]
item_row20 = [i for i in range(item)]
item_row21 = [i for i in range(item)]
item_dt10 = [i for i in range(item)]
item_dt11 = [i for i in range(item)]
item_dt20 = [i for i in range(item)]
item_dt21 = [i for i in range(item)]

for i in range(item):
    if i%1000==0:
        print(i)
    #対象アイテムに関連のあるレコードを抽出
    item_list11[i] = np.array(np.unique(np.where(item_id11==i)[0]), dtype="int")
    item_list12[i] = np.array(np.unique(np.where(item_id12==i)[0]), dtype="int")
    item_list2[i] = np.array(np.where(item_id2==i)[0], dtype="int")
    item_index1 = np.array(np.where(item_long1==i)[0], dtype="int")
    item_index2 = np.array(np.where(item_long2==i)[0], dtype="int")

    #対象アイテムと対象外アイテムのインデックスを取得
    item_record1 = np.unique(record_id1[item_index1])
    item_record2 = np.unique(record_id2[item_index2])
    row1 = np.array(np.where(np.in1d(record_id1, item_record1))[0], dtype="int")
    row2 = np.array(np.where(np.in1d(record_id2, item_record2))[0], dtype="int")
    target_index11 = np.array(np.where(item_long1[row1]==i)[0], dtype="int")
    target_index10 = np.delete(np.arange(row1.shape[0]), target_index11)
    target_index21 = np.array(np.where(item_long2[row2]==i)[0], dtype="int")
    target_index20 = np.delete(np.arange(row2.shape[0]), target_index21)

    #行列zとインデックスを格納
    z1 = np.array(Zi1[:, item_record1][row1, ].toarray(), dtype="int8")
    z2 = np.array(Zi2[:, item_record2][row2, ].toarray(), dtype="int8")
    item_dt10[i] = z1[target_index10, ]
    item_dt11[i] = z1[target_index11, ]
    item_dt20[i] = z2[target_index20, ]
    item_dt21[i] = z2[target_index21, ]
    item_row10[i] = row1[target_index10]
    item_row11[i] = row1[target_index11]
    item_row20[i] = row2[target_index20]
    item_row21[i] = row2[target_index21]

In [ ]:
##エレメントインデックスを作成
#インデックスの格納用配列
element_list11 = [i for i in range(element)]
element_list12 = [i for i in range(element)]
element_list2 = [i for i in range(element)]
element_row10 = [i for i in range(element)]
element_row11 = [i for i in range(element)]
element_row20 = [i for i in range(element)]
element_row21 = [i for i in range(element)]
element_dt10 = [i for i in range(element)]
element_dt11 = [i for i in range(element)]
element_dt20 = [i for i in range(element)]
element_dt21 = [i for i in range(element)]

for i in range(element):
    #対象エレメントに関連のあるレコードを抽出
    element_list11[i] = np.array(np.unique(np.where(element_id11==i)[0]), dtype="int")
    element_list12[i] = np.array(np.unique(np.where(element_id12==i)[0]), dtype="int")
    element_list2[i] = np.array(np.where(element_id2==i)[0], dtype="int")
    element_index1 = np.array(np.where(element_long1==i)[0], dtype="int")
    element_index2 = np.array(np.where(element_long2==i)[0], dtype="int")

    #対象エレメントの対象外アイテムのインデックスを取得
    element_record1 = np.unique(record_id1[element_index1])
    element_record2 = np.unique(record_id2[element_index2])
    row1 = np.array(np.where(np.in1d(record_id1, element_record1))[0], dtype="int")
    row2 = np.array(np.where(np.in1d(record_id2, element_record2))[0], dtype="int")
    target_index11 = np.array(np.where(element_long1[row1]==i)[0], dtype="int")
    target_index10 = np.delete(np.arange(row1.shape[0]), target_index11)
    target_index21 = np.array(np.where(element_long2[row2]==i)[0], dtype="int")
    target_index20 = np.delete(np.arange(row2.shape[0]), target_index21)

    #行列zとインデックスを格納
    z1 = Zi1[:, element_record1][row1, ]
    z2 = Zi2[:, element_record2][row2, ]
    element_dt10[i] = z1[target_index10, ]
    element_dt11[i] = z1[target_index11, ]
    element_dt20[i] = z2[target_index20, ]
    element_dt21[i] = z2[target_index21, ]
    element_row10[i] = row1[target_index10]
    element_row11[i] = row1[target_index11]
    element_row20[i] = row2[target_index20]
    element_row21[i] = row2[target_index21]

In [ ]:
####応答変数を生成####
##パラメータを生成
#事前分布を設定
alpha1 = np.array([0])
alpha21 = np.repeat(0.0, k1)
alpha22 = np.repeat(0.0, k2)
alpha23 = np.repeat(1.0, k3)
tau_u = np.array([0.4, 0.4, 0.6])
Cov_u = np.diag(np.repeat(0.2, k1))
Cov_v = np.diag(np.repeat(0.2, k2))
Cov_e = np.diag(np.repeat(0.1, k3))
tau = np.array([0.3, 0.3, 0.3])
taut_u = tau_u.copy(); taut = tau.copy()
Covt_u = Cov_u.copy(); Covt_v = Cov_v.copy(); Covt_e = Cov_e.copy()

#パラメータを生成
Sigma = np.array([0.4, 0.4, 1.0])
beta = np.array([-0.2, -0.2, 0.0])
beta_u11 = np.random.normal(0, tau_u[0], hh)
beta_u12 = np.random.normal(0, tau_u[1], hh)
beta_u2 = np.random.normal(0, tau_u[2], hh)
theta_u = np.random.multivariate_normal(alpha21, Cov_u, hh)
theta_v = np.random.multivariate_normal(alpha22, Cov_v, item)
theta_e = np.random.multivariate_normal(alpha23, Cov_e, element)
omega11 = np.random.normal(0, tau[0], k1*k2*k3).reshape(k2, k3, k1)
omega12 = np.random.normal(0, tau[1], k1*k2*k3).reshape(k2, k3, k1)
omega2 = np.random.normal(0, tau[2], k1*k2*k3).reshape(k2, k3, k1)
betat = beta.copy(); betat_u11 = beta_u11.copy(); betat_u12 = beta_u12.copy(); betat_u2 = beta_u2.copy()
thetat_u = theta_u.copy(); thetat_v = theta_v.copy(); thetat_e = theta_e.copy()
omegat11 = omega11.copy(); omegat12 = omega12.copy(); omegat2 = omega2.copy()
Sigmat = Sigma.copy()

##モデルの期待値を計算
#特徴行列のパラメータを拡張
theta_element11 = theta_e[element_long1, ]; theta_user11 = theta_u[d_long1, ]
theta_element12 = theta_e[element_long2, ]; theta_user12 = theta_u[d_long2, ]
theta_element2 = theta_e[element_id2, ]; theta_user2 = theta_u[d_id2, ]

#タッカー分解の期待値
uv_long11 = np.repeat(0.0, N1)
uv_long12 = np.repeat(0.0, N2)
uv2 = np.repeat(0.0, hhpt2)
for j in range(k1):
    uv_long11 += np.dot(np.dot(theta_v, omega11[:, :, j])[item_long1, ] * theta_element11 * theta_user11[:, j][:, np.newaxis], k_vec3)
    uv_long12 += np.dot(np.dot(theta_v, omega12[:, :, j])[item_long2, ] * theta_element12 * theta_user12[:, j][:, np.newaxis], k_vec3)
    uv2 += np.dot(np.dot(theta_v, omega2[:, :, j])[item_id2, ] * theta_element2 * theta_user2[:, j][:, np.newaxis], k_vec3)
uv11 = np.dot(sparse.csr_matrix(uv_long11), Zi1).toarray().reshape(-1)
uv12 = np.dot(sparse.csr_matrix(uv_long12), Zi2).toarray().reshape(-1)

#期待値を計算
mu1 = beta[0] + beta_u11[d_id1] + uv11
mu2 = beta[1] + beta_u12[d_id1] + uv12
mu3 = beta[2] + beta_u2[d_id2] + uv2 


##正規分布から連続応答変数と二値応答変数を生成
#連続応答変数を生成
y1 = np.random.normal(mu1, Sigma[0], hhpt1)
y2 = np.random.normal(mu2, Sigma[1], hhpt1)

#潜在効用から二値応答変数を生成
U = np.random.normal(mu3, Sigma[2], hhpt2); UT = U.copy()
y3 = (U >= 0)*1 + (U < 0)*0   #購買有無に変換
Prob = norm.cdf(mu3, 0, Sigma[2])

In [ ]:
#生成したデータのヒストグラム
dt = [y1, y2, Prob, uv11, uv12, uv2]
fig_range = np.append(np.array([np.min(y1), np.min(y2), 0, np.min(uv11), np.min(uv12), np.min(uv2)]), 
                      np.array([np.max(y1), np.max(y2), 1, np.max(uv11), np.max(uv12), np.max(uv2)])).reshape(2, len(dt))
colorlist = ["r", "g", "b", "c", "m", "y", "k", "w"]
legend = ["応答変数1の分布", "応答変数2の分布", "潜在確率の分布", "uv11の分布", "uv12の分布", "uv2の分布"]
fig = plt.figure(figsize=(13.0, 6.5))
for j in range(len(dt)):
    ax = fig.add_subplot(2, 3, j+1)
    ax.hist(dt[j],  bins=25, range=(fig_range[0, j], fig_range[1, j]), color=colorlist[j])
    plt.title(legend[j], fontsize=12.5)
fig.tight_layout()
plt.show()

In [ ]:
####マルコフ連鎖モンテカルロ法でGeneralized Factorization Machinesを推定####
##切断正規分布の乱数を発生させる関数
def rtnorm(mu, sigma, a, b, n):
    FA = norm.cdf(a, mu, sigma)
    FB = norm.cdf(b, mu, sigma)
    return norm.ppf(np.random.uniform(0, 1, n)*(FB-FA)+FA, mu, sigma)

In [ ]:
##ベイジアン多変量回帰モデルをギブスサンプリングする関数
def rmultireg(Y, X, inv_XXV, XY, Cov, ADelta, Deltabar, V, nu, n, col, k):
    #事後分布のパラメータを設定
    beta_mu = np.dot(inv_XXV, XY + np.dot(ADelta, Deltabar)).T.reshape(-1)   #平均ベクトル
    sigma = np.kron(Cov, inv_XXV)   #分散共分散行列

    #パラメータをサンプリング
    beta_vec = np.random.multivariate_normal(beta_mu, sigma, 1)
    beta = beta_vec.reshape(col, k, order='F')   #回帰行列に変換

    
    ##逆ウィシャート分布から分散共分散行列をサンプリング
    #モデル誤差を設定
    mu = np.dot(X, beta)
    er = Y - mu

    #逆ウィシャート分布のパラメータ
    IW_R = np.dot(er.T, er) + V
    Sn = n + nu

    #パラメータをサンプリング
    Cov = scipy.stats.invwishart.rvs(Sn, IW_R, 1)
    return beta, Cov

In [ ]:
##アルゴリズムの設定
R = 2000
keep = 2
burnin = int(100/keep)
iter = 0
disp = 10

In [ ]:
##データの設定
#切断領域を定義
rho = 100.0
a = np.array((1-y3)*(-rho) + y3*0)
b = np.array(y3*rho + (1-y3)*0)
flag =  y3 - (1-y3)

#特徴行列の割当インデックス
allocation_v = np.repeat(np.arange(k2), k3)
allocation_e = np.tile(np.arange(k3), k2)
allocation_omega = np.array([np.delete(np.arange(k1), j).tolist() for j in range(k1)])

#ユーザーのデータソースを1つに統合
hhpt = np.array([hhpt1, hhpt1, hhpt2])
pt = np.hstack((pt1[:, np.newaxis], pt1[:, np.newaxis], pt2[:, np.newaxis]))
d_list = [d_list1, d_list1, d_list2]
d_weight = pt / np.sum(pt, axis=1)[:, np.newaxis]

#アイテムのデータソースを1つに統合
item_n = np.hstack((np.array([item_list11[i].shape[0] for i in range(item)])[:, np.newaxis],
                    np.array([item_list12[i].shape[0] for i in range(item)])[:, np.newaxis],
                    np.array([item_list2[i].shape[0] for i in range(item)])[:, np.newaxis] ))
item_weight = item_n / np.sum(item_n, axis=1)[:, np.newaxis]

#エレメントのデータソースを1つに統合
element_n = np.hstack((np.array([element_list11[i].shape[0] for i in range(element)])[:, np.newaxis],
                       np.array([element_list12[i].shape[0] for i in range(element)])[:, np.newaxis],
                       np.array([element_list2[i].shape[0] for i in range(element)])[:, np.newaxis] ))
element_weight = element_n / np.sum(element_n, axis=1)[:, np.newaxis]

In [ ]:
##事前分布の設定
#階層モデルの事前分布
alpha1 = np.array([0.0])
alpha21 = np.repeat(0.0, k1)
alpha22 = np.repeat(0.0, k2)
alpha23 = np.repeat(1.0, k3)
tau = np.diag(np.repeat(0.1, k2*k3))
inv_tau = np.linalg.inv(tau)
V1 = 0.1 * np.diag(np.repeat(1.0, k1))
V2 = 0.1 * np.diag(np.repeat(1.0, k2))
V3 = 0.1 * np.diag(np.repeat(1.0, k3))
nu = 1
s01 = 0.1
v01 = 0.1

#モデルパラメータの事前分布
tau = 100 
s02 = 1.0
v02 = 1.0

In [ ]:
##パラメータの真値
#階層モデルのパラメータの真値
tau_u = taut_u.copy()
Cov_u = Covt_u.copy()
Cov_v = Covt_v.copy()
Cov_e = Covt_e.copy()
inv_Cov_u = np.linalg.inv(Cov_u)
inv_Cov_v = np.linalg.inv(Cov_v)
inv_Cov_e = np.linalg.inv(Cov_e)

#モデルパラメータの真値
Sigma = Sigmat.copy()
beta = betat.copy()
beta_u11 = betat_u11.copy()
beta_u12 = betat_u12.copy()
beta_u2 = betat_u2.copy()
theta_u = thetat_u.copy()
theta_v = thetat_v.copy()
theta_e = thetat_e.copy()
omega11 = omegat11.copy()
omega12 = omegat12.copy()
omega2 = omegat2.copy()

In [ ]:
##パラメータの初期値
#階層モデルのパラメータの初期値
tau_u = np.array([0.5, 0.5, 0.5])
Cov_u = np.diag(np.repeat(0.2, k1))
Cov_v = np.diag(np.repeat(0.2, k2))
Cov_e = np.diag(np.repeat(0.2, k3))
inv_Cov_u = np.linalg.inv(Cov_u)
inv_Cov_v = np.linalg.inv(Cov_v)
inv_Cov_e = np.linalg.inv(Cov_e)

#モデルパラメータの初期値
Sigma = np.array([0.3, 0.3, 1.0])
beta = np.repeat(0.0, s)
beta_u11 = np.random.normal(0, tau_u[0], hh)
beta_u12 = np.random.normal(0, tau_u[1], hh)
beta_u2 = np.random.normal(0, tau_u[2], hh)
theta_u = np.random.multivariate_normal(alpha21, Cov_u, hh)
theta_v = np.random.multivariate_normal(alpha22, Cov_v, item)
theta_e = np.random.multivariate_normal(alpha23, Cov_e, element)
omega11 = np.random.normal(0, 0.25, k1*k2*k3).reshape(k2, k3, k1)
omega12 = np.random.normal(0, 0.25, k1*k2*k3).reshape(k2, k3, k1)
omega2 = np.random.normal(0, 0.1, k1*k2*k3).reshape(k2, k3, k1)

In [ ]:
##モデルの期待値
#パラメータを拡張
beta_user11 = beta_u11[d_id1]
beta_user12 = beta_u12[d_id1]
beta_user2 = beta_u2[d_id2]
theta_element11 = theta_e[element_long1, ]; theta_user11 = theta_u[d_long1, ]
theta_element12 = theta_e[element_long2, ]; theta_user12 = theta_u[d_long2, ]
theta_element2 = theta_e[element_id2, ]; theta_user2 = theta_u[d_id2, ]

#タッカー分解の期待値
AR11 = np.zeros((k1, N1)); AR12 = np.zeros((k1, N2)); AR2 = np.zeros((hhpt2, k1))
uv_long11 = np.repeat(0.0, N1); uv_long12 = np.repeat(0.0, N2); uv2 = np.repeat(0.0, hhpt2)
for j in range(k1):
    ar11 = np.dot(theta_v, omega11[:, :, j])[item_long1, ] * theta_element11
    ar12 = np.dot(theta_v, omega12[:, :, j])[item_long2, ] * theta_element12
    ar2 = np.dot(theta_v, omega2[:, :, j])[item_id2, ] * theta_element2
    uv_long11 += np.dot(ar11 * theta_user11[:, j][:, np.newaxis], k_vec3)
    uv_long12 += np.dot(ar12 * theta_user12[:, j][:, np.newaxis], k_vec3)
    uv2 += np.dot(ar2 * theta_user2[:, j][:, np.newaxis], k_vec3)
    AR11[j, ] = np.dot(ar11, k_vec3)
    AR12[j, ] = np.dot(ar12, k_vec3)
    AR2[:, j] = np.dot(ar2, k_vec3)
    
#レコード単位で和を取る
uv11 = np.dot(sparse.csr_matrix(uv_long11), Zi1).toarray().reshape(-1)
uv12 = np.dot(sparse.csr_matrix(uv_long12), Zi2).toarray().reshape(-1)

#期待値を計算
mu1 = beta[0] + beta_user11 + uv11
mu2 = beta[1] + beta_user12 + uv12
mu3 = beta[2] + beta_user2 + uv2 

In [ ]:
##パラメータの格納用配列
#階層モデルのパラメータの格納用配列
TAU_U = np.zeros((int(R/keep), s))
COV_U = np.zeros((k1, k1, int(R/keep)))
COV_V = np.zeros((k2, k2, int(R/keep)))
COV_E = np.zeros((k3, k3, int(R/keep)))

#モデルパラメータの格納用配列
SIGMA = np.zeros((int(R/keep), s))
BETA = np.zeros((int(R/keep), s))
BETA_U11 = np.zeros((int(R/keep), hh))
BETA_U12 = np.zeros((int(R/keep), hh))
BETA_U2 = np.zeros((int(R/keep), hh))
THETA_U = np.zeros((hh, k1, int(R/keep)))
THETA_V = np.zeros((item, k2, int(R/keep)))
THETA_E = np.zeros((element, k3, int(R/keep)))
OMEGA11 = np.zeros((k2, k3, k1, int(R/keep)))
OMEGA12 = np.zeros((k2, k3, k1, int(R/keep)))
OMEGA2 = np.zeros((k2, k3, k1, int(R/keep)))

In [ ]:
##対数尤度の真値
#タッカー分解の期待値
uvt_long11 = np.repeat(0.0, N1); uvt_long12 = np.repeat(0.0, N2)
uvt2 = np.repeat(0.0, hhpt2)
for j in range(k1):
    ar11 = np.dot(thetat_v, omegat11[:, :, j])[item_long1, ] * thetat_e[element_long1, ]
    ar12 = np.dot(thetat_v, omegat12[:, :, j])[item_long2, ] * thetat_e[element_long2, ]
    ar2 = np.dot(thetat_v, omegat2[:, :, j])[item_id2, ] * thetat_e[element_id2, ]
    uvt_long11 += np.dot(ar11 * thetat_u[d_long1, j][:, np.newaxis], k_vec3)
    uvt_long12 += np.dot(ar12 * thetat_u[d_long2, j][:, np.newaxis], k_vec3)
    uvt2 += np.dot(ar2 * thetat_u[d_id2, j][:, np.newaxis], k_vec3)
    
#レコード単位で和を取る
uvt11 = np.dot(sparse.csr_matrix(uvt_long11), Zi1).toarray().reshape(-1)
uvt12 = np.dot(sparse.csr_matrix(uvt_long12), Zi2).toarray().reshape(-1)

#期待値を計算
mut1 = betat[0] + betat_u11[d_id1] + uvt11
mut2 = betat[1] + betat_u12[d_id1] + uvt12
mut3 = betat[2] + betat_u2[d_id2] + uvt2

#対数尤度を計算
LLbest11 = np.sum(scipy.stats.norm.logpdf(y1, mut1, Sigmat[0]))
LLbest12 = np.sum(scipy.stats.norm.logpdf(y2, mut2, Sigmat[1]))
Prob = norm.cdf(mut3, 0, Sigmat[2])
Prob[Prob==1.0] = 0.9999999999; Prob[Prob==0] = np.power(10.0, -100)
LLbest2 = np.sum(y3*np.log(Prob)) + np.sum((1-y3)*np.log(1-Prob))
LLbest = LLbest11 + LLbest12 + LLbest2
print(np.round([LLbest11, LLbest12, LLbest2, LLbest], 1))

In [ ]:
####ギブスサンプリングでパラメタータをサンプリング####
for rp in range(R):
    
    ##モデルの期待値と潜在効用の生成
    #期待値を計算
    mu1 = beta[0] + beta_user11 + uv11
    mu2 = beta[1] + beta_user12 + uv12
    mu3 = beta[2] + beta_user2 + uv2 

    #切断正規分布から潜在効用を生成
    U = rtnorm(mu3, Sigma[2], a, b, hhpt2)
    y = [y1, y2, U]


    ##モデルの期待値パラメータをサンプリング
    #モデル誤差を定義
    Sigma_sq = np.power(Sigma, 2)
    er_y1 = y1 - beta_user11 - uv11
    er_y2 = y2 - beta_user12 - uv12
    er_y3 = U - beta_user2 - uv2
    er_y = [er_y1, er_y2, er_y3]

    #正規分布から期待値をサンプリング
    weights = np.power(tau, 2) / (Sigma_sq/hhpt + np.power(tau, 2))
    mu_par = weights * np.array([np.mean(er_y[j]) for j in range(s)])
    beta = np.random.normal(mu_par, weights*Sigma_sq/hhpt, s)
    
    
    ##ランダム効果のパラメータをサンプリング
    #モデル誤差を定義
    er_y1 = y1 - beta[0] - uv11
    er_y2 = y2 - beta[1] - uv12
    er_y3 = U - beta[2] - uv2
    er_y = [er_y1, er_y2, er_y3]

    #正規分布からランダム効果をサンプリング
    for i in range(hh):
        weights = tau_u / (Sigma_sq/pt[i, ] + tau_u)
        mu_par = weights * np.array([np.mean(er_y[j][d_list[j][i]]) for j in range(s)])
        beta_u = np.random.normal(mu_par, weights*Sigma_sq/pt[i, ], s)
        beta_u11[i] = beta_u[0]
        beta_u12[i] = beta_u[1]
        beta_u2[i] = beta_u[2]

    #パラメータを更新
    beta_user11 = beta_u11[d_id1]
    beta_user12 = beta_u12[d_id1]
    beta_user2 = beta_u2[d_id2]


    ##ユーザーの特徴行列をサンプリング
    #モデル誤差と入力変数を定義
    X1 = np.dot(sparse.csr_matrix(AR11), Zi1).T.toarray()
    X2 = np.dot(sparse.csr_matrix(AR12), Zi2).T.toarray()
    er_y1 = y1 - beta[0] - beta_user11
    er_y2 = y2 - beta[1] - beta_user12
    er_y3 = U - beta[2] - beta_user2

    #多変量正規分布から事後分布をサンプリング
    for i in range(hh):
        index1 = d_list1[i]; index2 = d_list2[i]
        x = np.vstack((X1[index1, ], X2[index1, ], AR2[index2, ]))
        er_y = np.append(np.append(er_y1[index1], er_y2[index1]), er_y3[index2])
        xy = np.dot(x.T, er_y)
        xxv = np.dot(x.T, x) + inv_Cov_u
        inv_xxv = np.linalg.inv(xxv)
        theta_par = np.dot(inv_xxv, xy + np.dot(inv_Cov_u, alpha21))
        theta_u[i, ] = np.random.multivariate_normal(theta_par, np.sum(Sigma_sq*d_weight[i, ])*inv_xxv, 1)

    #パラメータを更新
    theta_user11 = theta_u[d_long1, ]
    theta_user12 = theta_u[d_long2, ]
    theta_user2 = theta_u[d_id2, ]


    ##アイテムの特徴行列をサンプリング
    #タッカー分解の期待値
    theta_item11 = theta_v[item_long1, ]; theta_item12 = theta_v[item_long2, ]; theta_item2 = theta_v[item_id2, ]
    AR11 = np.zeros((N1, k2)); AR12 = np.zeros((N2, k2)); AR2 = np.zeros((hhpt2, k2))
    uv_long11 = np.repeat(0.0, N1); uv_long12 = np.repeat(0.0, N2); uv2 = np.repeat(0.0, hhpt2)
    for j in range(k1):
        ar11 = np.dot(theta_e, omega11[:, :, j].T)[element_long1, ] * theta_user11[:, j][:, np.newaxis]
        ar12 = np.dot(theta_e, omega12[:, :, j].T)[element_long2, ] * theta_user12[:, j][:, np.newaxis]
        ar2 = np.dot(theta_e, omega2[:, :, j].T)[element_id2, ] * theta_user2[:, j][:, np.newaxis]
        uv_long11 += np.dot(ar11 * theta_item11, k_vec2)
        uv_long12 += np.dot(ar12 * theta_item12, k_vec2)
        uv2 += np.dot(ar2 * theta_item2, k_vec2)
        AR11 += ar11; AR12 += ar12; AR2 += ar2

    #データを転置
    AR11_T = AR11.T
    AR12_T = AR12.T

    #アイテムごとに特徴ベクトルをサンプリング
    for i in range(item):
        #インデックスの定義
        index11 = item_list11[i]; index12 = item_list12[i]; index2 = item_list2[i]
        row10 = item_row10[i]; row11 = item_row11[i]
        row20 = item_row20[i]; row21 = item_row21[i]

        #応答変数と入力変数の定義
        u11 = er_y1[index11] - np.dot(uv_long11[row10, ], item_dt10[i])
        u12 = er_y2[index12] - np.dot(uv_long12[row20, ], item_dt20[i])
        er_y = np.append(np.append(u11, u12), er_y3[index2])
        x11 = np.dot(AR11_T[:, row11], item_dt11[i]).T
        x12 = np.dot(AR12_T[:, row21], item_dt21[i]).T
        x = np.vstack((x11, x12, AR2[index2, ]))

        #多変量正規分布から事後分布をサンプリング
        xy = np.dot(x.T, er_y)
        xxv = np.dot(x.T, x) + inv_Cov_v
        inv_xxv = np.linalg.inv(xxv)
        theta_par = np.dot(inv_xxv, xy + np.dot(alpha22, inv_Cov_v))
        theta_v[i, ] = np.random.multivariate_normal(theta_par, np.sum(Sigma_sq*item_weight[i, ])*inv_xxv, 1)

    #パラメータの更新
    theta_item11 = theta_v[item_long1]
    theta_item12 = theta_v[item_long2, ]
    theta_item2 = theta_v[item_id2, ]


    ##エレメントの特徴行列をサンプリング
    #タッカー分解の期待値
    AR11 = np.zeros((N1, k3)); AR12 = np.zeros((N2, k3)); AR2 = np.zeros((hhpt2, k3))
    uv_long11 = np.repeat(0.0, N1); uv_long12 = np.repeat(0.0, N2); uv2 = np.repeat(0.0, hhpt2)
    for j in range(k1):
        ar11 = np.dot(theta_v, omega11[:, :, j])[item_long1, ] * theta_user11[:, j][:, np.newaxis]
        ar12 = np.dot(theta_v, omega12[:, :, j])[item_long2, ] * theta_user12[:, j][:, np.newaxis]
        ar2 = np.dot(theta_v, omega2[:, :, j])[item_id2, ] * theta_user2[:, j][:, np.newaxis]
        uv_long11 += np.dot(ar11 * theta_element11, k_vec3)
        uv_long12 += np.dot(ar12 * theta_element12, k_vec3)
        uv2 += np.dot(ar2 * theta_element2, k_vec3)
        AR11 += ar11; AR12 += ar12; AR2 += ar2

    #スパース行列に変換
    sparse_uv11 = sparse.csr_matrix(uv_long11)
    sparse_uv12 = sparse.csr_matrix(uv_long12)
    sparse_AR11 = sparse.csr_matrix(AR11).T
    sparse_AR12 = sparse.csr_matrix(AR12).T

    #エレメントごとに特徴ベクトルをサンプリング
    for i in range(element):
        #インデックスの定義
        index11 = element_list11[i]; index12 = element_list12[i]; index2 = element_list2[i]
        row10 = element_row10[i]; row11 = element_row11[i]
        row20 = element_row20[i]; row21 = element_row21[i]

        #応答変数と入力変数の定義
        u11 = er_y1[index11] - np.dot(sparse_uv11[:, row10], element_dt10[i]).toarray().reshape(-1)
        u12 = er_y2[index12] - np.dot(sparse_uv12[:, row20], element_dt20[i]).toarray().reshape(-1)
        er_y = np.append(np.append(u11, u12), er_y3[index2])
        x11 = np.dot(sparse_AR11[:, row11], element_dt11[i]).T.toarray()
        x12 = np.dot(sparse_AR12[:, row21], element_dt21[i]).T.toarray()
        x = np.vstack((x11, x12, AR2[index2, ]))

        #多変量正規分布から事後分布をサンプリング
        xy = np.dot(x.T, er_y)
        xxv = np.dot(x.T, x) + inv_Cov_e
        inv_xxv = np.linalg.inv(xxv)
        theta_par = np.dot(inv_xxv, xy + np.dot(alpha23, inv_Cov_e))
        theta_e[i, ] = np.random.multivariate_normal(theta_par, np.sum(Sigma_sq*element_weight[i, ])*inv_xxv, 1)

    #パラメータの更新
    theta_element11 = theta_e[element_long1, ]
    theta_element12 = theta_e[element_long2, ]
    theta_element2 = theta_e[element_id2, ]


    ##コアテンソルをサンプリング
    #タッカー分解の期待値を展開
    uv_deploy11 = np.zeros((N1, k1))
    uv_deploy12 = np.zeros((N2, k1))
    uv_deploy2 = np.zeros((hhpt2, k1))
    for j in range(k1):
        uv_deploy11[:, j] = np.dot(np.dot(theta_v, omega11[:, :, j])[item_long1, ] * 
                                   theta_element11 * theta_user11[:, j][:, np.newaxis], k_vec3)
        uv_deploy12[:, j] = np.dot(np.dot(theta_v, omega12[:, :, j])[item_long2, ] *
                                   theta_element12 * theta_user12[:, j][:, np.newaxis], k_vec3)
        uv_deploy2[:, j] = np.dot(np.dot(theta_v, omega2[:, :, j])[item_id2, ] * 
                                  theta_element2 * theta_user2[:, j][:, np.newaxis], k_vec3)

    #レコード単位で和を取る
    uv_deploy11 = np.dot(sparse.csr_matrix(uv_deploy11).T, Zi1).T.toarray()
    uv_deploy12 = np.dot(sparse.csr_matrix(uv_deploy12).T, Zi2).T.toarray()

    #特徴行列を全パターンに展開
    theta_allocation1 = theta_v[:, allocation_v]; theta_allocation2 = theta_e[:, allocation_e]
    theta_deploy11 = sparse.csr_matrix(theta_allocation1[item_long1, ] * theta_allocation2[element_long1, ])
    theta_deploy12 = sparse.csr_matrix(theta_allocation1[item_long2, ] * theta_allocation2[element_long2, ])
    theta_deploy2 = theta_allocation1[item_id2, ] * theta_allocation2[element_id2, ]
    sparse_u11 = sparse.csr_matrix(theta_user11)
    sparse_u12 = sparse.csr_matrix(theta_user12)

    for j in range(k1):
        #応答変数と入力変数の定義
        u11 = er_y1 - np.dot(uv_deploy11[:, allocation_omega[j, ]], np.repeat(1, k1-1))
        u12 = er_y2 - np.dot(uv_deploy12[:, allocation_omega[j, ]], np.repeat(1, k1-1))
        u2 = er_y3 - np.dot(uv_deploy2[:, allocation_omega[j, ]], np.repeat(1, k1-1))
        x11 = np.dot(theta_deploy11.multiply(sparse_u11[:, j]).T, Zi1).T.toarray()
        x12 = np.dot(theta_deploy12.multiply(sparse_u12[:, j]).T, Zi2).T.toarray()
        x2 = theta_deploy2 * theta_user2[:, j][:, np.newaxis]

        #事後分布をサンプリング
        xy11 = np.dot(x11.T, u11); xy12 = np.dot(x12.T, u12); xy2 = np.dot(x2.T, u2)
        xxv11 = np.dot(x11.T, x11) + inv_tau; xxv12 = np.dot(x12.T, x12) + inv_tau; xxv2 = np.dot(x2.T, x2) + inv_tau
        inv_xxv11 = np.linalg.inv(xxv11); inv_xxv12 = np.linalg.inv(xxv12); inv_xxv2 = np.linalg.inv(xxv2)
        omega_par11 = np.dot(inv_xxv11, xy11); omega_par12 = np.dot(inv_xxv12, xy12); omega_par2 = np.dot(inv_xxv2, xy2)
        omega11[:, :, j] = np.random.multivariate_normal(omega_par11, Sigma_sq[0]*inv_xxv11).reshape(k2, k3)
        omega12[:, :, j] = np.random.multivariate_normal(omega_par12, Sigma_sq[1]*inv_xxv12).reshape(k2, k3)
        omega2[:, :, j] = np.random.multivariate_normal(omega_par2, Sigma_sq[2]*inv_xxv2).reshape(k2, k3)
        
        
    ##モデルの期待値を更新
    #タッカー分解の期待値
    AR11 = np.zeros((k1, N1)); AR12 = np.zeros((k1, N2)); AR2 = np.zeros((hhpt2, k1))
    uv_long11 = np.repeat(0.0, N1); uv_long12 = np.repeat(0.0, N2); uv2 = np.repeat(0.0, hhpt2)
    for j in range(k1):
        ar11 = np.dot(theta_v, omega11[:, :, j])[item_long1, ] * theta_element11
        ar12 = np.dot(theta_v, omega12[:, :, j])[item_long2, ] * theta_element12
        ar2 = np.dot(theta_v, omega2[:, :, j])[item_id2, ] * theta_element2
        uv_long11 += np.dot(ar11 * theta_user11[:, j][:, np.newaxis], k_vec3)
        uv_long12 += np.dot(ar12 * theta_user12[:, j][:, np.newaxis], k_vec3)
        uv2 += np.dot(ar2 * theta_user2[:, j][:, np.newaxis], k_vec3)
        AR11[j, ] = np.dot(ar11, k_vec3)
        AR12[j, ] = np.dot(ar12, k_vec3)
        AR2[:, j] = np.dot(ar2, k_vec3)

    #レコード単位で和を取る
    uv11 = np.dot(sparse.csr_matrix(uv_long11), Zi1).toarray().reshape(-1)
    uv12 = np.dot(sparse.csr_matrix(uv_long12), Zi2).toarray().reshape(-1)

    #期待値を計算
    mu1 = beta[0] + beta_user11 + uv11
    mu2 = beta[1] + beta_user12 + uv12
    mu3 = beta[2] + beta_user2 + uv2 

    
    ##モデルの標準偏差をサンプリング
    #逆ガンマ分布のパラメータ
    er1 = y1 - mu1; er2 = y2 - mu2
    s1 = np.dot(er1, er1) + s02
    s2 = np.dot(er2, er2) + s02
    v1 = hhpt1 + v02

    #パラメータをサンプリング
    Sigma[0] = np.sqrt(1/np.random.gamma(v1/2, 1/(s1/2), 1))
    Sigma[1] = np.sqrt(1/np.random.gamma(v1/2, 1/(s2/2), 1))


    ##ランダム効果の標準偏差をサンプリング
    #逆ガンマ分布のパラメータ
    er11 = beta_u11 - np.mean(beta_u11); er12 = beta_u12 - np.mean(beta_u12); er2 = beta_u2 - np.mean(beta_u2)
    s11 = np.dot(er11, er11) + s01
    s12 = np.dot(er12, er12) + s01
    s2 = np.dot(er2, er2) + s01
    v1 = hh + v01

    #パラメータをサンプリング
    tau_u[0] = np.sqrt(1/np.random.gamma(v1/2, 1/(s11/2), 1))
    tau_u[1] = np.sqrt(1/np.random.gamma(v1/2, 1/(s12/2), 1))
    tau_u[2] = np.sqrt(1/np.random.gamma(v1/2, 1/(s2/2), 1))


    ##特徴行列の分散共分散行列をサンプリング
    #逆ウィシャート行列のパラメータ
    er_u = theta_u - np.mean(theta_u, axis=0)
    er_v = theta_v - np.mean(theta_v, axis=0)
    IW_R1 = np.dot(er_u.T, er_u) + V1
    IW_R2 = np.dot(er_v.T, er_v) + V2
    Sn1 = hh + nu
    Sn2 = item + nu

    #パラメータをサンプリング
    Cov_u = np.diag(np.diag(scipy.stats.invwishart.rvs(Sn1, IW_R1, 1)))
    Cov_v = np.diag(np.diag(scipy.stats.invwishart.rvs(Sn2, IW_R2, 1)))
    inv_Cov_u = np.linalg.inv(Cov_u)
    inv_Cov_v = np.linalg.inv(Cov_v)
    
    
    ##サンプリング結果の格納と表示
    if rp%keep==0:
        #階層モデルのパラメータのサンプリング結果の格納
        mkeep = int(rp/keep)
        TAU_U[mkeep, ] = tau_u
        COV_U[:, :, mkeep] = Cov_u
        COV_V[:, :, mkeep] = Cov_v

        #モデルパラメータのサンプリング結果を格納
        SIGMA[mkeep, ] = Sigma
        BETA[mkeep, ] = beta
        BETA_U11[mkeep, ] = beta_u11
        BETA_U12[mkeep, ] = beta_u12
        BETA_U2[mkeep, ] = beta_u2
        THETA_U[:, :, mkeep] = theta_u
        THETA_V[:, :, mkeep] = theta_v
        THETA_E[:, :, mkeep] = theta_e
        OMEGA11[:, :, :, mkeep] = omega11
        OMEGA12[:, :, :, mkeep] = omega12
        OMEGA2[:, :, :, mkeep] = omega2

    #対数尤度の更新とサンプリング結果を表示
    if rp%disp==0:
        #学習データの対数尤度を更新
        LL11 = np.sum(scipy.stats.norm.logpdf(y1, mu1, Sigma[0]))
        LL12 = np.sum(scipy.stats.norm.logpdf(y2, mu2, Sigma[1]))
        Prob = norm.cdf(mu3, 0, Sigma[2])
        Prob[Prob==1.0] = 0.9999999999; Prob[Prob==0] = np.power(10.0, -100)
        LL2 = np.sum(y3*np.log(Prob)) + np.sum((1-y3)*np.log(1-Prob))
        LL = LL11 + LL12 + LL2

        #サンプリング結果の表示
        print(rp)
        print(["推定された対数尤度", np.round(LL11, 1), np.round(LL12, 1), np.round(LL2, 1), np.round(LL, 1)])
        print(["真値の対数尤度", np.round(LLbest11, 1), np.round(LLbest12, 1), np.round(LLbest2, 1), np.round(LLbest, 1)])
        print(["モデルの平均二乗誤差"] + np.round(np.append(Sigma, Sigmat), 3).tolist())